<a href="https://colab.research.google.com/github/camille-310/depot-UA/blob/main/04_pyspark_movie_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark MovieLens


## 0. Setup (Run this cell)

In [2]:
!pip install -q pyspark

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("PySpark_Student_Exercises_Movies")
    .master("local[*]")
    .getOrCreate()
)

spark

## 1. Download & Load MovieLens Dataset

In [3]:
!wget -q https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -q ml-latest-small.zip

ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies  = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)

## Task 1 — Data Exploration

**1.1 Inspect the data**

*	Show first 10 rows of ratings
*	Print schema for both DataFrames
*	Count the number of rows in each




**1.2 Column selection**

Write PySpark code to:

* select only userId, movieId, rating
*	rename movieId → film_id
*	cast rating to integer or float explicitly

**1.3 Filtering**

Using ratings:

*	Find all ratings from userId = 1
*	Find all ratings greater than 4.5
*	Find all ratings on movieId in (1, 50, 100)


(use isin())

**1.4 Sorting & limiting**

*	Show the top 20 highest ratings
*	Show the 10 lowest ratings made by user 600
*	Sort by rating desc and timestamp asc

**1.5 Derived columns**

Create columns:

*	rating_x2 = rating * 2
*	positive_rating = 1 if rating ≥ 4 else 0
*	log_rating = log10(rating + 1)

**1.6 Missing values**

Even if ML-latest-small has no nulls, :

*	Add a fake null column
*	Demonstrate fill/replace/drop operations

**1.7 Distinct & deduplication**

*	Count unique users
*	Count unique movies rated
*	Drop duplicate ratings where (userId, movieId) might repeat (even though dataset is clean)

**1.1 Inspect the data**

*	Show first 10 rows of ratings
*	Print schema for both DataFrames
*	Count the number of rows in each

In [4]:
# Write your solution here
ratings.show(10)

print("schema ratings :", ratings.schema)
print("schema movies :", movies.schema)

total_rows_ratings = ratings.count()
print("Number of rows ratings:", total_rows_ratings)

total_rows_movies = movies.count()
print("Number of rows movies:", total_rows_movies)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows

schema ratings : StructType([StructField('userId', IntegerType(), True), StructField('movieId', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('timestamp', IntegerType(), True)])
schema movies : StructType([StructField('movieId', IntegerType(), True), StructField('title', StringType(), True), StructField('genres', StringType(), True)])
Number of rows ratings: 100836
Number of rows movies: 9742


**1.2 Column selection**

Write PySpark code to:

* select only userId, movieId, rating
*	rename movieId → film_id
*	cast rating to integer or float explicitly


In [5]:
from pyspark.sql.functions import col

ratings12 = ratings.select(col("userId"),
               col("movieId").alias("film_id"),
               col("rating").cast("float"))

ratings12.show(5)

+------+-------+------+
|userId|film_id|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



**1.3 Filtering**

Using ratings:

*	Find all ratings from userId = 1
*	Find all ratings greater than 4.5
*	Find all ratings on movieId in (1, 50, 100)

(use isin())

In [6]:
ratings.filter(col("userId")=="1").show(5)
ratings.filter(col("rating")>4.5).show(5)
ratings.filter(col("movieId").isin("1", "50", "100")).show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|    101|   5.0|964980868|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 5 rows

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|     50|   5.0|964982931|
|     5|      1|   4.0|847434962|
|     5|     50|   4.0|847434881|
|     6|     50|   1.0|845553381|
+------+-------+------+---------+
only showing top 5 rows



****1.4 Sorting & limiting**

*	Show the top 20 highest ratings
*	Show the 10 lowest ratings made by user 600
*	Sort by rating desc and timestamp asc**

In [7]:
ratings.orderBy(col("rating").desc()).show(20)
ratings.filter(col("userId")=="600").orderBy(col("rating").asc()).show(10)
ratings.orderBy(col("rating").desc(), col("timestamp").asc()).show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    954|   5.0|964983219|
|     1|   1220|   5.0|964981909|
|     1|   1023|   5.0|964982681|
|     1|     47|   5.0|964983815|
|     1|   1024|   5.0|964982876|
|     1|    101|   5.0|964980868|
|     1|   1025|   5.0|964982791|
|     1|    157|   5.0|964984100|
|     1|   1029|   5.0|964982855|
|     1|    216|   5.0|964981208|
|     1|   1031|   5.0|964982653|
|     1|    260|   5.0|964981680|
|     1|   1032|   5.0|964982791|
|     1|    457|   5.0|964981909|
|     1|   1049|   5.0|964982400|
|     1|    553|   5.0|964984153|
|     1|   1073|   5.0|964981680|
|     1|    608|   5.0|964982931|
|     1|   1080|   5.0|964981327|
|     1|    919|   5.0|964982475|
+------+-------+------+---------+
only showing top 20 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|   600|    762|   0.5|1237707803|
|   600|   8665|  

**1.5 Derived columns**

Create columns:

*	rating_x2 = rating * 2
*	positive_rating = 1 if rating ≥ 4 else 0
*	log_rating = log10(rating + 1)

In [8]:
from pyspark.sql.functions import lit, expr, when

ratings15 = ratings.withColumn("rating_x2",col("rating")*2)\
      .withColumn("positive_rating",when(col("rating") >= 4, 1).otherwise(0))\
      .withColumn("log_rating",expr("log10(rating + 1)"))

ratings15.show(5)

+------+-------+------+---------+---------+---------------+------------------+
|userId|movieId|rating|timestamp|rating_x2|positive_rating|        log_rating|
+------+-------+------+---------+---------+---------------+------------------+
|     1|      1|   4.0|964982703|      8.0|              1|0.6989700043360189|
|     1|      3|   4.0|964981247|      8.0|              1|0.6989700043360189|
|     1|      6|   4.0|964982224|      8.0|              1|0.6989700043360189|
|     1|     47|   5.0|964983815|     10.0|              1|0.7781512503836436|
|     1|     50|   5.0|964982931|     10.0|              1|0.7781512503836436|
+------+-------+------+---------+---------+---------------+------------------+
only showing top 5 rows



**1.6 Missing values**

Even if ML-latest-small has no nulls, :

*	Add a fake null column
*	Demonstrate fill/replace/drop operations

In [13]:
from pyspark.sql.types import StringType

ratings16 = ratings.withColumn("fake",lit(None).cast(StringType()))
ratings16.show(5)
ratings16 = ratings16.na.fill({"fake": "Unknown"})
ratings16.show(5)
ratings16 = ratings16.replace("Unknown", "Pas renseigné", subset=["fake"])
ratings16.show(5)
ratings16 = ratings16.drop("fake")
ratings16.show(5)

+------+-------+------+---------+----+
|userId|movieId|rating|timestamp|fake|
+------+-------+------+---------+----+
|     1|      1|   4.0|964982703|NULL|
|     1|      3|   4.0|964981247|NULL|
|     1|      6|   4.0|964982224|NULL|
|     1|     47|   5.0|964983815|NULL|
|     1|     50|   5.0|964982931|NULL|
+------+-------+------+---------+----+
only showing top 5 rows

+------+-------+------+---------+-------+
|userId|movieId|rating|timestamp|   fake|
+------+-------+------+---------+-------+
|     1|      1|   4.0|964982703|Unknown|
|     1|      3|   4.0|964981247|Unknown|
|     1|      6|   4.0|964982224|Unknown|
|     1|     47|   5.0|964983815|Unknown|
|     1|     50|   5.0|964982931|Unknown|
+------+-------+------+---------+-------+
only showing top 5 rows

+------+-------+------+---------+-------------+
|userId|movieId|rating|timestamp|         fake|
+------+-------+------+---------+-------------+
|     1|      1|   4.0|964982703|Pas renseigné|
|     1|      3|   4.0|964981

**1.7 Distinct & deduplication**

*	Count unique users
*	Count unique movies rated
*	Drop duplicate ratings where (userId, movieId) might repeat (even though dataset is clean)

In [11]:
print("unique userID :", ratings.select("userId").distinct().count())
print("unique movies rated :", ratings.filter(ratings.movieId.isNotNull()).select("movieId").distinct().count())
ratings17 = ratings.dropDuplicates(["userId", "movieId"])
ratings17.show(10)

unique userID : 610
unique movies rated : 9724
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|   1208|   4.0| 964983250|
|     1|   1348|   4.0| 964983393|
|     4|    457|   5.0| 945079259|
|     4|    599|   2.0| 945078587|
|     6|    274|   4.0| 845555151|
|     6|    327|   1.0| 845554062|
|     6|    520|   3.0| 845553844|
|     7|   1101|   4.0|1106635502|
|     8|    235|   3.0| 839464076|
|     9|   5481|   5.0|1044656836|
+------+-------+------+----------+
only showing top 10 rows



## Task 2 — Aggregations & GroupBy

**2.1 Simple aggregations**

*	Compute min, max, avg rating
*	Count total number of ratings
*	Count ratings per movieId

**2.2 GroupBy**

Compute:

*	average rating per movie
*	number of ratings per movie
*	average rating per user
*	number of ratings per user

**2.3 Top items**

*	Top 20 most-rated movies
*	Top 20 best-rated movies (min 50 ratings)

→ You must filter with a join or window

**2.4 Window functions**

Using Window partitioned by movieId:

*	rank users by timestamp (earliest → latest rating)
*	create a lag column: previous rating by same user
*	compute average rating per movie with window


In [ ]:
# Write your solution here

**2.1 Simple aggregations**

*	Compute min, max, avg rating
*	Count total number of ratings
*	Count ratings per movieId

In [28]:
from pyspark.sql.functions import count, round, sum, avg, min, max, countDistinct, approx_count_distinct

ratings.select(min("rating").alias("min_rating")).show()
ratings.select(max("rating").alias("max_rating")).show()
ratings.select(round(avg("rating"),3).alias("avg_rating")).show()

ratings.select(count("rating").alias("total_ratings")).show()

ratings.groupBy("movieId").agg(count("rating").alias("total_ratings")).show(10)

+----------+
|min_rating|
+----------+
|       0.5|
+----------+

+----------+
|max_rating|
+----------+
|       5.0|
+----------+

+----------+
|avg_rating|
+----------+
|     3.502|
+----------+

+-------------+
|total_ratings|
+-------------+
|       100836|
+-------------+

+-------+-------------+
|movieId|total_ratings|
+-------+-------------+
|   1580|          165|
|   2366|           25|
|   3175|           75|
|   1088|           42|
|  32460|            4|
|  44022|           23|
|  96488|            4|
|   1238|            9|
|   1342|           11|
|   1591|           26|
+-------+-------------+
only showing top 10 rows



**2.2 GroupBy**

Compute:

*	average rating per movie
*	number of ratings per movie
*	average rating per user
*	number of ratings per user


**2.3 Top items**

*	Top 20 most-rated movies
*	Top 20 best-rated movies (min 50 ratings)

→ You must filter with a join or window


**2.4 Window functions**

Using Window partitioned by movieId:

*	rank users by timestamp (earliest → latest rating)
*	create a lag column: previous rating by same user
*	compute average rating per movie with window

## Task 3 — Spark SQL

In [ ]:
ratings.createOrReplaceTempView("ratings_table")
movies.createOrReplaceTempView("movies_table")

**3.1 Simple SQL Queries**

*	Show first 20 rows
*	Count total ratings
*	Count distinct users
*	Average rating overall

**3.2 SQL Grouping**
*	Average rating by movie
*	Ratings count by movie
*	Best movies with at least 100 ratings
*	Worst movies with at least 100 ratings
*	Most active users

**3.3 SQL CASE WHEN**

Create rating buckets:

*	≥ 4.0 → “high”
*	3.0–3.9 → “medium”
*	< 3.0 → “low”

**3.4 SQL Window Functions**

*	Top 10 movies by rating for each genre
*	Earliest rating per user (row_number)
*	Rolling average rating per movie

In [ ]:
# Write your solution here

## Task 4 — Joins & Genre Analytics

**4.1 Basic join**

Join ratings → movies on movieId.

*	Show 20 joined rows
*	Show userId, title, rating
*	Count how many ratings each genre has

**4.2 Parse genres**

genres looks like "Action|Adventure|Sci-Fi"

*	split genres into an array
*	explode the genres
*	count ratings per genre
*	compute average rating per genre

**4.3 Join + Aggregation**

Compute:

*	average rating per genre
*	average rating per movie title
*	number of ratings per movie
*	number of ratings per genre per year (bonus: extract year from title)

**4.4 Left Anti Join**

Find:

*	movies in movies.csv with no ratings
*	number of such movies

In [ ]:
# Write your solution here

## Task 5 — MLlib Classification

**Goal: Build a binary classifier:**

Predict whether a user will give rating ≥ 4.0

5.1 Create label

Add a column:

In [ ]:
label = 1 if rating >= 4 else 0

**5.2 Feature engineering**

Create features:

	•	rating (as-is)
	•	timestamp
	•	normalized timestamp
	•	(optional) number of ratings by that user (using join or window)

Use VectorAssembler to pack them.

**5.3 Split data**

70% train, 30% test.

**5.4 Train model**

Train a Logistic Regression model.

	•	print coefficients
	•	print intercept
	•	print ROC AUC

**5.5 Evaluate**

Compute :

	•	accuracy
	•	precision
	•	recall
	•	confusion matrix (TP, FP, TN, FN)

**5.6 Task: Improve model**

try:

	•	Adding a log-transformed timestamp
	•	Using a DecisionTreeClassifier
	•	Using a RandomForestClassifier
	•	Comparing metrics

In [ ]:
# Write your solution here

## Task 6 — Performance & Execution

**6.1 Check partitions**

Get number of partitions for ratings and movies.

**6.2 Repartition and coalesce**

	•	explain difference
	•	show effect on shuffles
	•	check number of partitions with rdd.getNumPartitions()

**6.3 Cache**

Cache ratings:

	•	compute count
	•	compute average rating per movie twice
	•	measure difference using Python time

**6.4 explain(True)**

run explain() on:

	•	a join
	•	a groupBy
	•	a window function

And identify shuffle boundaries.

In [ ]:
# Write your solution here

## Stop Spark

In [ ]:
spark.stop()